In [26]:
import numpy as np
from sklearn.metrics import classification_report
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Sequential

import cv2 as cv

### Reading data

In [46]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  'data/dataset/',
  label_mode='categorical',
  shuffle=True,
  seed=42,
  image_size=(32, 32),
)

test_ds = tf.keras.utils.image_dataset_from_directory(
  'data/test_dataset/',
  label_mode='categorical',
  shuffle=True,
  seed=42,
  image_size=(32, 32),
)

train_X, train_y = zip(*train_ds)
train_X = np.concatenate(train_X)
train_y = np.concatenate(train_y)

test_X, test_y = zip(*test_ds)
test_X = np.concatenate(test_X)
test_y = np.concatenate(test_y)


Found 10019 files belonging to 10 classes.
Found 353 files belonging to 10 classes.


### Gray

In [61]:
def cvt_to_gray(images):
    gray_images = []
    for rgb_image in images:
        gray_image = cv.cvtColor(rgb_image, cv.COLOR_RGB2GRAY)
        gray_images.append(gray_image)
    gray_images = np.array(gray_images)
    gray_images = gray_images[..., np.newaxis]
    return gray_images

In [62]:
gray_train_X = cvt_to_gray(train_X)
gray_test_X = cvt_to_gray(test_X)
gray_train_X, gray_test_X

### Gradients

In [86]:
def get_gradients(rgb_image):
    gray_image = cv.cvtColor(rgb_image, cv.COLOR_RGB2GRAY)
    
    sobelx = cv.Sobel(gray_image, cv.CV_64F,1,0,ksize=3)
    sobely = cv.Sobel(gray_image, cv.CV_64F,0,1,ksize=3)

    return [sobelx, sobely]

In [87]:
def cvt_to_gradients(images):
    grad_images = []
    for rgb_image in images:
        gradients = get_gradients(rgb_image)
        grad_images.append(gradients)
    grad_images = np.array(grad_images)
    grad_images = np.transpose(grad_images, (0, 2, 3, 1))
    return grad_images

In [88]:
grad_train_X = cvt_to_gradients(train_X)
grad_test_X = cvt_to_gradients(test_X)
grad_train_X.shape, grad_test_X.shape

((10019, 32, 32, 2), (353, 32, 32, 2))

In [93]:
model = Sequential([
    layers.Input(shape=(None, None, 2)),
    layers.Resizing(32, 32),
    layers.Rescaling(1./255),
    layers.RandomFlip('horizontal'),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax'),
])

In [94]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_9 (Resizing)       (None, 32, 32, 2)         0         
                                                                 
 rescaling_9 (Rescaling)     (None, 32, 32, 2)         0         
                                                                 
 random_flip_9 (RandomFlip)  (None, 32, 32, 2)         0         
                                                                 
 conv2d_27 (Conv2D)          (None, 30, 30, 32)        608       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 15, 15, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_28 (Conv2D)          (None, 13, 13, 64)        18496     
                                                      

In [95]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [96]:
history = model.fit(grad_train_X, train_y, validation_data=(grad_test_X, test_y), epochs=40, batch_size=64)

Epoch 1/40
157/157 [==============================] - 6s 34ms/step - loss: 2.0933 - accuracy: 0.2113 - val_loss: 2.2407 - val_accuracy: 0.2351
Epoch 2/40
157/157 [==============================] - 5s 29ms/step - loss: 1.3433 - accuracy: 0.5192 - val_loss: 2.0131 - val_accuracy: 0.3598
Epoch 3/40
157/157 [==============================] - 5s 31ms/step - loss: 0.8714 - accuracy: 0.7044 - val_loss: 2.1089 - val_accuracy: 0.3909
Epoch 4/40
157/157 [==============================] - 5s 32ms/step - loss: 0.6023 - accuracy: 0.8003 - val_loss: 3.1782 - val_accuracy: 0.3088
Epoch 5/40
157/157 [==============================] - 5s 32ms/step - loss: 0.4359 - accuracy: 0.8532 - val_loss: 2.5113 - val_accuracy: 0.3768
Epoch 6/40
157/157 [==============================] - 5s 30ms/step - loss: 0.3059 - accuracy: 0.8992 - val_loss: 4.8239 - val_accuracy: 0.2663
Epoch 7/40
157/157 [==============================] - 5s 32ms/step - loss: 0.2236 - accuracy: 0.9284 - val_loss: 3.2095 - val_accuracy: 0.4079

In [22]:
X, y = zip(*test_ds)
X = np.concatenate(X)
y = np.concatenate(y)

In [203]:
X.shape, y.shape

((72, 32, 32, 3), (72, 10))

In [69]:
X, y = zip(*test_ds)
X = np.concatenate(X)
y = np.concatenate(y)

predictions = model.predict(X)
report = classification_report(np.argmax(y, axis=1), np.argmax(predictions, axis=1))
print(report)

ValueError: in user code:

    File "c:\Users\Lukasz\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\Lukasz\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Lukasz\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\Lukasz\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "c:\Users\Lukasz\anaconda3\envs\tf\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Lukasz\anaconda3\envs\tf\lib\site-packages\keras\engine\input_spec.py", line 277, in assert_input_compat